In [58]:
from elasticsearch import Elasticsearch, helpers
import regex as re
from subprocess import Popen, PIPE, STDOUT
import os
import pandas as pd
import re

from tqdm.notebook import tqdm

import numpy as np

tqdm.pandas(desc="Progress")


In [59]:
es_server = Elasticsearch(
    hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)


<ipython-input-59-09391bbada40>:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es_server = Elasticsearch(


In [60]:
es_server.ping()


True

In [61]:
!curl -X PUT "http://localhost:9200/_snapshot/es_repo?pretty" -H 'Content-Type: application/json' -d '{"type": "fs", "settings": { "location": "/usr/share/elasticsearch/es_repo",  "compress": true} }'


{
  "acknowledged" : true
}


In [62]:
es_server.snapshot.get_repository(name="es_repo")


ObjectApiResponse({'es_repo': {'type': 'fs', 'settings': {'compress': 'true', 'location': '/usr/share/elasticsearch/es_repo'}}})

In [133]:
!curl -X GET "http://localhost:9200/_snapshot/es_repo/_all?pretty"


{
  "snapshots" : [
    {
      "snapshot" : "geonames_snapshot_demonyms",
      "uuid" : "iDpznuFNTluOIGDIS5t0yA",
      "repository" : "es_repo",
      "version_id" : 7140299,
      "version" : "7.14.2",
      "indices" : [
        ".geoip_databases",
        "geonames",
        "demonyms"
      ],
      "data_streams" : [ ],
      "include_global_state" : false,
      "state" : "SUCCESS",
      "start_time" : "2022-11-11T10:17:58.621Z",
      "start_time_in_millis" : 1668161878621,
      "end_time" : "2022-11-11T10:19:15.250Z",
      "end_time_in_millis" : 1668161955250,
      "duration_in_millis" : 76629,
      "failures" : [ ],
      "shards" : {
        "total" : 3,
        "failed" : 0,
        "successful" : 3
      },
      "feature_states" : [ ]
    }
  ]
}


In [64]:
!curl -X POST "http://localhost:9200/_snapshot/es_repo/geonames_snapshot_denonyms/_restore?pretty&wait_for_completion=true" -H 'Content-Type: application/json' -d '{ "ignore_unavailable": true, "include_global_state": false }'


{
  "snapshot" : {
    "snapshot" : "geonames_snapshot_denonyms",
    "indices" : [
      "geonames",
      "denonyms"
    ],
    "shards" : {
      "total" : 2,
      "failed" : 0,
      "successful" : 2
    }
  }
}


In [65]:
!curl -XGET "http://localhost:9200/_cat/indices?pretty"


green  open .geoip_databases LsYDpLtdT7-wLP64-8CPFQ 1 0       41 0 39.3mb 39.3mb
yellow open denonyms         gkgGtaNxRIKV4ConlPP-Nw 1 1      261 3 88.6kb 88.6kb
yellow open geonames         R-WzMbwsROenRyXTOp9WoA 1 1 12081476 0  2.9gb  2.9gb


In [66]:
def get_coordinates_es(loc_name, exact_matching=True):
    txt = loc_name.lower()
    if exact_matching:
        query = {
                "bool": {
                    "should": [
                        {"term": {"name2": txt}},
                        {"term": {"asciiname2": txt}},
                        {"term": {"alternatenames2": txt}}
                    ]
                }
            }
    else:
        query = {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": txt,
                            "fields": ['name', 'asciiname', 'alternatenames'],
                            "operator":   "and"
                        }
                    }
                }
            }
    result = []
    for query_result in es_server.search(index="geonames", query=query, size=50)["hits"]["hits"]:
        source = query_result["_source"]
        result.append((source["latitude"], source["longitude"], source.get(
            "population", 0), source.get("feature_code", "W")))
    result.sort(key=lambda candidate: candidate[2], reverse=True)
    return result


In [67]:
get_coordinates_es("Brazil")


[(-10.0, -55.0, 209469333, 'PCLI'),
 (39.52365, -87.12501999999999, 8109, 'PPLA2'),
 (1.63414, -75.5719, 0, 'PPL'),
 (34.0265, -90.28065, 0, 'PPL'),
 (35.91757, -89.06257, 0, 'PPL'),
 (14.32438, 120.9784, 0, 'ST'),
 (10.55471, -61.275690000000004, 0, 'PPL'),
 (40.758070000000004, -92.95437, 0, 'W'),
 (40.758070000000004, -92.95437, 0, 'PPL'),
 (37.524809999999995, -84.04159, 0, 'PPL'),
 (37.84616, -91.06347, 0, 'PPL'),
 (-27.96667, 30.066670000000002, 0, 'FRM'),
 (-29.816670000000002, 17.133329999999997, 0, 'LCTY'),
 (-29.84057, 17.12048, 0, 'FRM'),
 (12.64933, -86.2379, 0, 'PPL'),
 (12.26063, -85.92436, 0, 'PPL'),
 (33.522659999999995, 36.2819, 0, 'DIP')]

In [119]:
def get_full_name_es(demonym, exact_matching=True):
    txt = demonym.lower()
    if exact_matching: 
      query = {
        "query": {
          "bool": {
            "should": [
              { "term": { "adjectivals_lower": txt }},
              { "term": { "demonyms_lower": txt }}
            ]
          }
        }
      }
    else:
      query = {
        "query": {
          "bool": {
            "must": {
              "multi_match": {
                "query": txt,
                "fields": ['adjectivals', 'demonyms'],
                "operator":   "and"
              }
            }
          }
        }
      }
    result = []
    for query_result in es_server.search(index="demonyms", body=query, size=50)["hits"]["hits"]:
        source = query_result["_source"]
        result.append((source["entity_name"], source["geonames_id"], source["geonames_name"], source["adjectivals"], source["demonyms"]))
    return result

In [ ]:
get_full_name_es("Sahrawi", False)

<ipython-input-112-4aa5c5815717>:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  for query_result in es_server.search(index="denonyms", body=query, size=50)["hits"]["hits"]:


[('Western Sahara',
  2461445,
  'Western Sahara',
  ['Sahrawi', ' Sahrawian', ' Sahraouian'],
  ['Sahrawis', ' Sahraouis'])]